In [1]:
# # Prepare Google Colab Environement and build handmade library
# !git clone https://github.com/kaenova/Headline_Detection.git
# %cd "/content/Headline_Detection"

# !make lib

# %cd "/content/"

# print("Please upload '4. Processed.zip'")
# from google.colab import files
# files.upload()

# !unzip "/content/4. Processed.zip"

# print("Please upload .env")
# from google.colab import files
# files.upload()

In [2]:
# # Reset Google Colab Environment
# %cd ..
# !rm -fr Headline_Detection

# Load Config and Env

In [2]:
# Load env for S3 uploads
from dotenv import load_dotenv
from kaelib.s3 import check_required_env_vars
import os
load_dotenv()
check_required_env_vars()

# Load Dataset

In [12]:
import os
import pandas as pd

environment = 'local'
if environment == 'local':
    data_path = "../../data/4. Processed/"
elif environment == 'colab':
    data_path = "/content/4. Processed/"
else:
    raise ImportError("Data environment")


folders = os.listdir(data_path)
scenario_data = {}

for folder in folders:
    current_data_path = f"{data_path}/{folder}"
    if not os.path.isdir(current_data_path):
        continue
    files = os.listdir(current_data_path)
    data_dict = {}
    for file in files:
        current_file = f"{current_data_path}/{file}"
        data_dict[file] = pd.read_csv(current_file)
    scenario_data[int(folder)] = data_dict

In [13]:
import torch
import pandas
import typing
from torch.utils.data import Dataset
from kaelib.processor import TextProcessingPipeline


class TextClassificationDataset(Dataset):
    def __init__(
        self,
        df: "pandas.DataFrame",
        x_column_name: "str" = "tweet",
        y_column_name: "str" = "labels",
        preprocessor: "typing.Optional[TextProcessingPipeline]" = None,
    ):
        self.x = df[x_column_name].astype(str).to_list()
        self.y = torch.tensor(df[y_column_name].astype(int).to_list())
        assert len(self.x) == len(self.y)
        self.preprocessor = preprocessor

    def __len__(self):
        return len(self.x)

    def _process_idx_text(self, idx):
        data = self.x[idx]
        if type(idx) is not slice:
            data = [self.x[idx]]
        if self.preprocessor is not None:
            data = self.preprocessor.process_corpus(data)
        return data

    def __getitem__(self, idx):
        processed_corpus = self._process_idx_text(idx)
        return processed_corpus, self.y[idx]

    def __repr__(self) -> str:
        return "\n".join([f"{self.x[i]} : {self.y[i]}" for i in range(5)])


scenario_datasets = {}
for scenario in scenario_data:
    data = scenario_data[scenario]
    datasets = {}
    for data_type in data:
        datasets[data_type] = TextClassificationDataset(data[data_type])
    scenario_datasets[scenario] = datasets


# Modelling

In [20]:
# Hyperparameters
hyper_params = {
    'seq_length': 256,
    'learning_rate': 1e-5,
    'batch_size': 32
}

In [26]:
# https://arxiv.org/pdf/1607.01759.pdf

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchnlp.word_to_vector import FastText
import lightning.pytorch as pl
from torchmetrics.classification import F1Score, Accuracy, Recall, Precision


class FastTextClassifier(pl.LightningModule):
    def __init__(
        self,
        seq_length: "int" = 256,
        out_feature: "int" = 2,
        pad_sequence: "bool" = False,
    ):
        super().__init__()
        self.seq_length = seq_length
        self.pad_sequence = pad_sequence
        self.fasttext = FastText("id")
        self.feed_forward = nn.Sequential(
            nn.Linear(300, 150),
            nn.ReLU(),
            nn.Linear(150, 64),
            nn.ReLU(),
            nn.Linear(64, out_feature),
        )
        self.f1_scorer = F1Score(task="multiclass", num_classes=out_feature)
        self.accuracy_scorer = Accuracy(task="multiclass", num_classes=out_feature)
        self.precision_scorer = Precision(task="multiclass", num_classes=out_feature)
        self.recall_scorer = Recall(task="multiclass", num_classes=out_feature)

    def _forward_fasttext(self, x: "list[str]"):
        batch_text_embedding = torch.tensor([]).to(self.device)
        for sentence in x:
            sentence_seq = sentence.split(" ")
            if len(sentence_seq) > self.seq_length:
                sentence_seq = sentence_seq[: self.seq_length]
            if self.pad_sequence:
                while len(sentence_seq) < 256:
                    sentence_seq.append("<pad>")
            word_embedding = (
                self.fasttext[sentence_seq].mean(dim=0).unsqueeze(0).to(self.device)
            )
            batch_text_embedding = torch.cat((batch_text_embedding, word_embedding))
        return batch_text_embedding

    def forward(self, x: "list[str]") -> "torch.Tensor":
        # Prepare str
        logits: torch.Tensor = self._forward_fasttext(x)
        logits = self.feed_forward(logits)
        return logits

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        pred = self.forward(x[0])
        loss = F.cross_entropy(pred, y)
        self.log("training_loss", loss)
        f1_score = self.f1_scorer(pred, y)
        self.log("training_f1", f1_score, prog_bar=True)
        accuracy = self.accuracy_scorer(pred, y)
        self.log("training_accuracy", accuracy, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        pred = self.forward(x[0])
        loss = F.cross_entropy(pred, y)
        self.log("validation_loss", loss)
        f1_score = self.f1_scorer(pred, y)
        self.log("validation_f1", f1_score, prog_bar=True)
        accuracy = self.accuracy_scorer(pred, y)
        self.log("validation_accuracy", accuracy, prog_bar=True)
        precision = self.precision_scorer(pred, y)
        self.log("validation_precision", precision, prog_bar=True)
        recall = self.recall_scorer(pred, y)
        self.log("validation_recall", recall, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        pred = self.forward(x[0])
        loss = F.cross_entropy(pred, y)
        self.log("test_loss", loss)
        f1_score = self.f1_scorer(pred, y)
        self.log("test_f1", f1_score, prog_bar=True)
        accuracy = self.accuracy_scorer(pred, y)
        self.log("test_accuracy", accuracy, prog_bar=True)
        precision = self.precision_scorer(pred, y)
        self.log("test_precision", precision, prog_bar=True)
        recall = self.recall_scorer(pred, y)
        self.log("test_recall", recall, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=hyper_params['learning_rate'])
        return optimizer


In [28]:
model_module = FastTextClassifier
def create_model():
    return model_module(hyper_params['seq_length'], 2)

In [29]:
model = create_model()
model(['test','halo']).shape

torch.Size([2, 2])

# Train, Val, Test

In [30]:
available_cpus = 0
model_name = "fasttext"

In [31]:
import datetime
import random
import torch
import numpy as np

from torch.utils.data import DataLoader
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint

now = datetime.datetime.now()
timestamp = now.strftime('%Y-%m-%d_%H-%M-%S')

for scenario in scenario_datasets:
    np.random.seed(2023)
    random.seed(2023)
    torch.manual_seed(2023)

    run_id = f"{timestamp}_scenario_{scenario}"

    if scenario in [0]:
        continue

    model = create_model()

    # Dataset
    train_dataset = scenario_datasets[scenario]["train.csv"]
    test_dataset = scenario_datasets[scenario]["test.csv"]
    validation_dataset = scenario_datasets[scenario]["validation.csv"]
    train_loader = DataLoader(train_dataset, batch_size=hyper_params['batch_size'], num_workers=available_cpus)
    validation_loader = DataLoader(validation_dataset, batch_size=hyper_params['batch_size'], num_workers=available_cpus)
    test_loader = DataLoader(test_dataset, batch_size=hyper_params['batch_size'], num_workers=available_cpus)

    tb_log = TensorBoardLogger("tensorboard", model_name, run_id)
    tb_log.log_hyperparams(hyper_params)

    checkpoint_callback_val = ModelCheckpoint(monitor="validation_loss", filename='val_{epoch}-{validation_loss:.4f}')
    checkpoint_callback_train = ModelCheckpoint(monitor="training_loss", filename='train_{epoch}-{training_loss:.4f}')
    early_stop_callback = EarlyStopping(
        monitor="validation_loss", min_delta=1e-3, patience=5, mode="min"
    )

    trainer = pl.Trainer(
        logger=tb_log,
        log_every_n_steps=2, 
        callbacks=[early_stop_callback, checkpoint_callback_val, checkpoint_callback_train]
    )
    trainer.fit(
        model=model, train_dataloaders=train_loader, val_dataloaders=validation_loader
    )
    model = model_module.load_from_checkpoint(checkpoint_callback_val.best_model_path)
    trainer.test(model=model, dataloaders=test_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name             | Type                | Params
---------------------------------------------------------
0 | feed_forward     | Sequential          | 54.9 K
1 | f1_scorer        | MulticlassF1Score   | 0     
2 | accuracy_scorer  | MulticlassAccuracy  | 0     
3 | precision_scorer | MulticlassPrecision | 0     
4 | recall_scorer    | MulticlassRecall    | 0     
---------------------------------------------------------
54.9 K    Trainable params
0         Non-trainable params
54.9 K    Total params
0.220     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]


KeyboardInterrupt



In [ ]:
id2label = {
 0 : "Bukan headline",
 1 : "Headline"
}

test_input = [
    "aku suka aplikasi ini", 
    "tidak suka sama aplikasi ini", 
    "oke", 
    "keren aplikasi", 
    "pengungkapan itu ditandai dengan ditangkapnya 4 orang pria, dan ditemukan sabu dalam klip plastik bening saat digeledah"
]
with torch.no_grad():
    pred = model(test_input)
    pred = F.softmax(pred, dim=1)
    pred_np = pred.argmax(dim=1).cpu().detach().numpy()
    for i in range(len(test_input)):
        print(f"'{test_input[i]}' : {id2label[pred_np[i]]}")

# Upload Logging and Checkpoint to S3

In [32]:
import os
from kaelib.s3 import upload_folder_to_s3
s3_folders = os.path.join(os.getenv("BUCKET_FOLDER"), model_name)
upload_folder_to_s3(f"tensorboard/{model_name}", s3_folders)



Uploading:   0%|                                                                             | 0/100 [00:00<?, ?file/s]

Uploading:   0%| | 0/100 [00:00<?, ?file/s, file=2023-04-12_11-47-02_scenario_1\events.out.tfevents.1681274823.PCRumah.

Uploading:   1%| | 1/100 [00:00<00:25,  3.83file/s, file=2023-04-12_11-47-02_scenario_1\events.out.tfevents.1681274823.

Uploading:   1%| | 1/100 [00:00<00:25,  3.83file/s, file=2023-04-12_11-47-02_scenario_1\events.out.tfevents.1681274919.

Uploading:   2%|▍                  | 2/100 [00:00<00:25,  3.83file/s, file=2023-04-12_11-47-02_scenario_1\hparams.yaml]

Uploading:   3%|▌                  | 3/100 [00:00<00:12,  7.53file/s, file=2023-04-12_11-47-02_scenario_1\hparams.yaml]

Uploading:   3%| | 3/100 [00:01<00:12,  7.53file/s, file=2023-04-12_11-47-02_scenario_1\checkpoints\train_epoch=356-tra

Uploading:   4%| | 4/100 [00:01<00:33,  2.84file/s, file=2023-04-12_11-47-02_scenario_1\checkpoints\train_epoch=356-tra

Uploading:   4%| | 4/100 [00:0

Upload complete!
